# *Dependências*

**Instalação de Dependências e declaração de Importações.** 

```
# No primeiro uso do dia, acionar esses comandos

```



In [ ]:
!pip install dtale
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install fsspec
!pip install string-grouper

import pandas as pd
import numpy as np
import missingno as msno

import dtale
import dtale.app as dtale_app


dtale_app.USE_COLAB = True


#dtale.show(df)

# Casos, Óbitos e Recuperados

**Casos e Óbitos por COVID-19 por Data e por Município** *(principal fonte de dados para casos, óbitos e recuperados para os municípios. Inclui informações geográficas para criar mapas)*

In [ ]:
import pandas as pd
import numpy as np


#localização dos dataframes
url1 = 'https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv' 
url2 = 'https://raw.githubusercontent.com/SoSoJigsaw/Carcara/main/meso_micro_regioes_sp.csv'


#leitura dos arquivos csv pelos pandas
covidsp = pd.read_csv(url1, sep=';') 
regiaosp = pd.read_csv(url2) 


# Agrupando outro dataframe ao principal, para inserir informações de regiões
regiaosp_tmp = regiaosp.drop(['latitude', 'municipio', 'longitude'], axis=1)
df = covidsp.merge(regiaosp_tmp, left_on=['codigo_ibge'], 
                   right_on=['codigo_ibge'], how='inner', 
                   suffixes=['_covidsp', '_regiaosp'])


# Data Cleaning

# Renomeando colunas
df.rename(columns={"nome_munic": "Município"}, inplace=True)
df.rename(columns={"datahora": "Data"}, inplace=True)
df.rename(columns={"area": "Área (Km2)"}, inplace=True)
df.rename(columns={"latitude": "Latitude"}, inplace=True)
df.rename(columns={"longitude": "Longitude"}, inplace=True)
df.rename(columns={"casos_mm7d": "Casos por mm7d"}, inplace=True)
df.rename(columns={"obitos_mm7d": "Óbitos por mm7d"}, inplace=True)
df.rename(columns={"letalidade": "Letalidade"}, inplace=True)
df.rename(columns={"casos": "Total de Casos"}, inplace=True)
df.rename(columns={"casos_novos": "Novos Casos"}, inplace=True)
df.rename(columns={"obitos": "Total de Óbitos"}, inplace=True)
df.rename(columns={"obitos_novos": "Novos Óbitos"}, inplace=True)
df.rename(columns={"mesorregiao": "Mesorregião"}, inplace=True)
df.rename(columns={"microrregiao": "Microrregião"}, inplace=True)
df.rename(columns={"pop": "População"}, inplace=True)


# Deletando colunas sem dados relevantes
df.drop('dia', axis=1, inplace=True)
df.drop('mes', axis=1, inplace=True)
df.drop('nome_drs', axis=1, inplace=True)
df.drop('cod_drs', axis=1, inplace=True)
df.drop('map_leg', axis=1, inplace=True)
df.drop('map_leg_s', axis=1, inplace=True)
df.drop('semana_epidem', axis=1, inplace=True)
df.drop('nome_ra', axis=1, inplace=True)
df.drop('cod_ra', axis=1, inplace=True)
df.drop('pop_60', axis=1, inplace=True)
df.drop('casos_pc', axis=1, inplace=True)
df.drop('obitos_pc', axis=1, inplace=True)


# Mudando a formatação do registro de data
df.loc[:, 'Data'] = pd.Series(pd.to_datetime
                                  (df['Data'], infer_datetime_format=True), 
                                  name='Data', index=df['Data'].index)                                  


# Alterando a "," pelo "." para a conversão em float sem gerar valores NaN
df['Latitude'] = df['Latitude'].str.replace(',','.')
df['Longitude'] = df['Longitude'].str.replace(',','.')
df['Casos por mm7d'] = df['Casos por mm7d'].str.replace(',','.')
df['Óbitos por mm7d'] = df['Óbitos por mm7d'].str.replace(',','.')
df['Letalidade'] = df['Letalidade'].str.replace(',','.')


# Corrigindo o formato da área para "km2" sem casas decimais
df['Área (Km2)'] /= 100
df['Área (Km2)'] = df['Área (Km2)'].round(decimals=0).astype(pd.Int64Dtype())


# Alterando o Datatype de algumas colunas
df['Latitude'] = df['Latitude'].apply(pd.to_numeric, downcast='float', 
                                      errors='coerce')
df['Longitude'] = df['Longitude'].apply(pd.to_numeric, downcast='float', 
                                        errors='coerce')
df['Casos por mm7d'] = df['Casos por mm7d'].apply(pd.to_numeric, downcast='float', 
                                          errors='coerce')
df['Óbitos por mm7d'] = df['Óbitos por mm7d'].apply(pd.to_numeric, downcast='float', 
                                            errors='coerce')
df['Letalidade'] = df['Letalidade'].apply(pd.to_numeric, downcast='float', 
                                          errors='coerce')


# Delimitando as casas decimais em colunas float:
df['Casos por mm7d'] = df['Casos por mm7d'].round(decimals=2)
df['Óbitos por mm7d'] = df['Óbitos por mm7d'].round(decimals=2)
df['Letalidade'] = df['Letalidade'].round(decimals=2)


# Excluindo valores ditos "ignorados" pela fonte dos dados
df.drop(df[df.Município == 'Ignorado'].index, inplace=True)
df.dropna().reset_index(drop=True, inplace=True)


# Excluindo linhas nas quais todos os dados variáveis são NaN (45.194 linhas)
df = df.loc[(df['Total de Casos'] + df['Novos Casos'] + df['Total de Óbitos'] + 
             df['Novos Óbitos'] != 0)]


# Forçando a ordenação por data e por fim resetando o index
df = df.sort_values(['Data'], ascending=True)

df = df[[c for c in df.columns if c not in ['index']]]
if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
	df = df.to_frame(index=False)
df = df.reset_index().drop('index', axis=1, errors='ignore')


df

,Município,codigo_ibge,Data,Total de Casos,Novos Casos,Casos por mm7d,Total de Óbitos,Novos Óbitos,Óbitos por mm7d,Letalidade,População,Área (Km2),Latitude,Longitude,Mesorregião,Microrregião
0,São Paulo,3550308,2020-02-25,1,0,0.00,0,0,0.00,0.00,11869660,1521,-23.532900,-46.639500,Metropolitana de São Paulo,São Paulo
1,São Paulo,3550308,2020-02-26,1,0,0.00,0,0,0.00,0.00,11869660,1521,-23.532900,-46.639500,Metropolitana de São Paulo,São Paulo
2,São Paulo,3550308,2020-02-27,1,0,0.00,0,0,0.00,0.00,11869660,1521,-23.532900,-46.639500,Metropolitana de São Paulo,São Paulo
3,São Paulo,3550308,2020-02-28,2,1,0.00,0,0,0.00,0.00,11869660,1521,-23.532900,-46.639500,Metropolitana de São Paulo,São Paulo
4,São Paulo,3550308,2020-02-29,2,0,0.00,0,0,0.00,0.00,11869660,1521,-23.532900,-46.639500,Metropolitana de São Paulo,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335351,Cachoeira Paulista,3508603,2021-10-06,2744,1,1.43,44,0,0.00,0.02,32231,288,-22.666500,-45.015400,Vale do Paraíba Paulista,Guaratinguetá
335352,Pontalinda,3540259,2021-10-06,209,0,0.00,15,0,0.00,0.07,4487,210,-20.439600,-50.525799,São José do Rio Preto,Jales
335353,Pontal,3540200,2021-10-06,5129,0,0.00,163,0,0.14,0.03,49283,356,-21.021601,-48.042301,Ribeirão Preto,Ribeirão Preto
335354,Pongaí,3540101,2021-10-06,358,0,0.00,11,0,0.00,0.03,3351,18,-21.739599,-49.360401,Bauru,Bauru


**Casos e Óbitos do Estado de São Paulo por Data** *(Principal fonte de dados do Estado, facilitará os gráficos sem a necessidade de fazer concatenação das colunas dos dados sobre os Municípios)*

In [69]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import sys
import time
import glob
import os
import datetime as dt
from datetime import datetime


# Automação da coleta de um dos dataframes
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
download_dir = "/content/covid_estado"
prefs = {'download.default_directory' : download_dir}

options_chrome = Options()
options_chrome.add_argument("start-maximized")
options_chrome.add_argument("disable-infobars")
options_chrome.add_argument("--disable-extensions")
options_chrome.add_argument('--headless')
options_chrome.add_argument('--no-sandbox')
options_chrome.add_argument('--disable-dev-shm-usage')
options_chrome.add_experimental_option('prefs', prefs)
desired = options_chrome.to_capabilities()
desired['loggingPrefs'] = { 'performance': 'ALL'}

navegador = webdriver.Chrome('chromedriver', options=options_chrome, 
                             desired_capabilities=desired)

navegador.get("https://www.seade.gov.br/coronavirus/#")
elemento = navegador.find_element_by_xpath("/html/body/div[1]/div[1]/a[1]")
href = elemento.get_attribute("href")
navegador.get(href)

time.sleep(20)


# Automação do reconhecimento de novos dados pelo Pandas
list_of_files = glob.glob('/content/covid_estado/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
csv = os.path.abspath(latest_file)

url = 'https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/sp.csv'


# Leitura pelo Pandas do arquivo csv
df1 = pd.read_csv(csv, sep=';', encoding='latin1')
df2 = pd.read_csv(url, sep=';')


# Agrupando os dois dataframes que se completam nas informações
df1 = df1.drop(['Data', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 
                'Unnamed: 8'], axis=1)
df = df2.merge(df1, left_on=['casos_acum'], right_on=['Total de casos'], 
               how='inner', suffixes=['_1', '_2'])


# Data Cleaning

# Deletando colunas duplicadas
df.drop('Total de casos', axis=1, inplace=True)


# Renomeando colunas
df.rename(columns={"casos_acum": "Total de casos"}, inplace=True)
df.rename(columns={"obitos_acum": "Total de óbitos"}, inplace=True)
df.rename(columns={"datahora": "Data"}, inplace=True)


# Mudando a formatação do registro de data para datetime64
df.loc[:, 'Data'] = pd.Series(pd.to_datetime(df['Data'], 
                                             infer_datetime_format=True), 
                                  name='Data', index=df['Data'].index)                                  


# Padronizando valores "0" errôneamente descritos como NaN
df["Óbitos por dia"] = df["Óbitos por dia"].fillna(0)


# Alterando o Datatype da coluna "Óbitos por dia" para int64 
df["Óbitos por dia"] = df["Óbitos por dia"].astype(int)


# Excluindo dados que estavam duplicadas
df = df.sort_values(['Data'], ascending=True)

df.rename(columns={"Óbitos por dia": "obitos"}, inplace=True)
df.rename(columns={"Casos por dia": "casos"}, inplace=True)

df = df.drop(df[(df.obitos == 0) & (df.casos == 0) & 
                (df.Data < '2021-01-01')].index)

df.rename(columns={"obitos": "Óbitos por dia"}, inplace=True)
df.rename(columns={"casos": "Casos por dia"}, inplace=True)


# Forçando a ordenação por data e por fim resetando o index
df = df.sort_values(['Data'], ascending=True)

df = df[[c for c in df.columns if c not in ['index']]]
if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
	df = df.to_frame(index=False)
df = df.reset_index().drop('index', axis=1, errors='ignore')


,Data,Total de casos,Total de óbitos,Casos por dia,Óbitos por dia
0,2020-02-26,1,0,1,0
1,2020-02-27,1,0,1,0
2,2020-02-28,1,0,1,0
3,2020-02-29,2,0,1,0
4,2020-03-01,2,0,1,0
...,...,...,...,...,...
585,2021-10-03,4369538,150064,477,14
586,2021-10-04,4369743,150086,205,22
587,2021-10-05,4373508,150229,3765,143
588,2021-10-06,4375187,150401,1679,172


# SRAG-Covid


**Pacientes internados por SRAG** - *Base de dados com registros individualizados de internações por Síndrome Respiratório Aguda Grave, ocasionadas ou não por COVID-19 (Para o trabalho, os casos relacionados à COVID-19 é que interessam, já que servem para demonstrar o real panorama da Pandemia)*

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import sys
import time
import glob
import os
import datetime as dt
from datetime import datetime


# Automação da coleta dos dataframes
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
download_dir = "/content/srag"
prefs = {'download.default_directory' : download_dir}

options_chrome = Options()
options_chrome.add_argument("start-maximized")
options_chrome.add_argument("disable-infobars")
options_chrome.add_argument("--disable-extensions")
options_chrome.add_argument('--headless')
options_chrome.add_argument('--no-sandbox')
options_chrome.add_argument('--disable-dev-shm-usage')
options_chrome.add_experimental_option('prefs', prefs)
desired = options_chrome.to_capabilities()
desired['loggingPrefs'] = { 'performance': 'ALL'}

navegador = webdriver.Chrome('chromedriver', options=options_chrome, 
                             desired_capabilities=desired)

navegador.get("https://www.saopaulo.sp.gov.br/planosp/simi/dados-abertos/")
botao1 = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[5]/h3/span").click()
elemento = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[5]/div/ul/li[1]/div/a")
href = elemento.get_attribute("href")
navegador.get(href)

time.sleep(330)


# Automação do reconhecimento de novos dados pelo Pandas
list_of_files = glob.glob('/content/srag/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
csv = os.path.abspath(latest_file)


# Leitura pelo Pandas do arquivo csv
df = pd.read_csv(csv, sep=';')


# Cleaning Data

# Excluindo dados referentes a outros Estados
df.rename(columns={"Sg Uf": "UF"}, inplace=True)
df.drop(df[df.UF != 'SP'].index, inplace=True)


# Excluindo dados referentes a casos de SRAG alheios à covid-19
df.rename(columns={"OUTRAS SRAG": "Doença"}, inplace=True)
df.drop(df[df.Doença != 'COVID 19'].index, inplace=True)


# Deletando colunas sem dados relevantes
df.drop('UF', axis=1, inplace=True)
df.drop('Última Data de Notificação', axis=1, inplace=True)
df.drop('Cs Raca', axis=1, inplace=True)
df.drop('Cs Sexo', axis=1, inplace=True)
df.drop('Quantidade Homens', axis=1, inplace=True)
df.drop('Quantidade Mulheres', axis=1, inplace=True)
df.drop('Quantidade de Casos', axis=1, inplace=True)
df.drop('Cs Raca_ID', axis=1, inplace=True)
df.drop('Cs Sexo_ID', axis=1, inplace=True)
df.drop('Faixa etária', axis=1, inplace=True)
df.drop('EVOLUCAO_NOT_NULL', axis=1, inplace=True)


# Renomeando colunas
df.rename(columns={"Nu Idade N": "Idade do paciente"}, inplace=True)
df.rename(columns={"Grupo de Idades": "Faixa Etária"}, inplace=True)
df.rename(columns={"Municípios": "Município"}, inplace=True)


# Ajustando valores que estão em caixa alta
s = df['Município'].str.title()
df.loc[:, 'Município'] = pd.Series(s, name='Município')
s = df['Doença'].str.title()
df.loc[:, 'Doença'] = pd.Series(s, name='Doença')


# Transformando a coluna de Data de Notificação em Datetime type
df.loc[:, 'Data de Notificação'] = pd.Series(pd.to_datetime(df['Data de Notificação'], 
                                                            dayfirst=True, format='%d/%m/%Y', 
                                                            errors='coerce'), 
                                             name='Data de Notificação')


# Alterando a " " pelo "-" na coluna "Doença"
df['Doença'] = df['Doença'].str.replace(' ','-')


# Padronizando Missing Values encontrados na coluna "Evolução"
df = df.fillna(value='Não informado') 


# Ordenando a coluna "Data de Notificação" e resetando o index
df = df.reset_index(drop=True)

df = df.sort_values(['Data de Notificação'], ascending=True)

df = df[[c for c in df.columns if c not in ['index']]]
if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
	df = df.to_frame(index=False)
df = df.reset_index().drop('index', axis=1, errors='ignore')


,Município,Idade do paciente,Faixa Etária,Doença,Evolução,Data de Notificação
0,Aruja,40,40,Covid-19,Cura,2020-01-02
1,Indaiatuba,46,45,Covid-19,Óbito,2020-01-02
2,Braganca Paulista,55,55,Covid-19,Cura,2020-01-05
3,Osasco,68,65,Covid-19,Cura,2020-01-07
4,Santo Andre,22,20,Covid-19,Cura,2020-01-08
...,...,...,...,...,...,...
522762,Sao Paulo,72,70,Covid-19,Em avaliação,2021-10-05
522763,Barretos,72,70,Covid-19,Óbito,2021-10-05
522764,Guararema,77,75,Covid-19,Em avaliação,2021-10-05
522765,Sao Paulo,76,75,Covid-19,Em avaliação,2021-10-05


# Vacinação

**Vacinômetro SP** *(é a fonte principal para os dados de vacinação dos municípios e do Estado)*

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import sys
import os
import glob
import time


# Automação da coleta dos dataframes pelo selenium

# Primeiro dataframe
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
download_dir = "/content/vacinometro"
prefs = {'download.default_directory' : download_dir}

options_chrome = Options()
options_chrome.add_argument("start-maximized")
options_chrome.add_argument("disable-infobars")
options_chrome.add_argument("--disable-extensions")
options_chrome.add_argument('--headless')
options_chrome.add_argument('--no-sandbox')
options_chrome.add_argument('--disable-dev-shm-usage')
options_chrome.add_experimental_option('prefs', prefs)
desired = options_chrome.to_capabilities()
desired['loggingPrefs'] = { 'performance': 'ALL'}

navegador = webdriver.Chrome('chromedriver', options=options_chrome, 
                             desired_capabilities=desired)

navegador.get("https://www.saopaulo.sp.gov.br/planosp/simi/dados-abertos/")
botao1 = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[14]").click()
elemento1 = navegador.find_element_by_css_selector("article.hrf-entry:nth-child(14) > div:nth-child(2) > ul:nth-child(2) > li:nth-child(1) > div:nth-child(1) > a:nth-child(2)")
href1 = elemento1.get_attribute("href")
navegador.get(href1)

# Segundo dataframe
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
download_dir2 = "/content/distribuicao"
prefs2 = {'download.default_directory' : download_dir2}

options_chrome2 = Options()
options_chrome2.add_argument("start-maximized")
options_chrome2.add_argument("disable-infobars")
options_chrome2.add_argument("--disable-extensions")
options_chrome2.add_argument('--headless')
options_chrome2.add_argument('--no-sandbox')
options_chrome2.add_argument('--disable-dev-shm-usage')
options_chrome2.add_experimental_option('prefs', prefs2)
desired = options_chrome2.to_capabilities()
desired['loggingPrefs'] = { 'performance': 'ALL'}

navegador2 = webdriver.Chrome('chromedriver', options=options_chrome2, 
                             desired_capabilities=desired)

navegador2.get("https://www.saopaulo.sp.gov.br/planosp/simi/dados-abertos/")
botao2 = navegador2.find_element_by_xpath("/html/body/section[4]/div/div/article[15]/h3").click()
elemento2 = navegador2.find_element_by_xpath("/html/body/section[4]/div/div/article[15]/div/ul/li/div/a")
href2 = elemento2.get_attribute("href")
navegador2.get(href2)

time.sleep(30)


# Automatação do reconhecimento de novos dados pelo Pandas
list_of_files1 = glob.glob('/content/vacinometro/*.csv')
latest_file1 = max(list_of_files1, key=os.path.getctime)
csv1 = os.path.abspath(latest_file1)

list_of_files2 = glob.glob('/content/distribuicao/*.csv')
latest_file2 = max(list_of_files2, key=os.path.getctime)
csv2 = os.path.abspath(latest_file2)


# Leitura pelo Pandas dos arquivos csv
vacinometro = pd.read_csv(csv1, sep=';')
distribuicao = pd.read_csv(csv2, sep=';')


# Cleaning Data

# Criando novas colunas de tal forma que não tenha linhas duplicadas
col = 'Dose'
condition1 = [vacinometro[col] == '1° DOSE']
condition2 = [vacinometro[col] == '2° DOSE']
condition3 = [vacinometro[col] == '3º DOSE']
condition4 = [vacinometro[col] == 'UNICA']
choice = [vacinometro["Total Doses Aplicadas"]]
                       
vacinometro["1ª Dose"] = np.select(condition1, choice, default=np.nan)
vacinometro["2ª Dose"] = np.select(condition2, choice, default=np.nan)
vacinometro["3ª Dose"] = np.select(condition3, choice, default=np.nan)
vacinometro["Dose Única"] = np.select(condition4, choice, default=np.nan)

vacinometro = vacinometro.groupby('Município').agg({'1ª Dose':'first', 
                                                    '2ª Dose':'first', 
                                                    '3ª Dose':'first', 
                                                    'Dose Única':'first'}).reset_index()


# Ajustando os 5 missing values encontrados para converter em int64:
vacinometro["3ª Dose"] = vacinometro["3ª Dose"].fillna(0)
vacinometro["Dose Única"] = vacinometro["Dose Única"].fillna(0)


# Ajustando o Datatype de colunas
vacinometro["1ª Dose"] = vacinometro["1ª Dose"].astype(int)
vacinometro["2ª Dose"] = vacinometro["2ª Dose"].astype(int)
vacinometro["3ª Dose"] = vacinometro["3ª Dose"].astype(int)
vacinometro["Dose Única"] = vacinometro["Dose Única"].astype(int)


# Alterando nome de colunas
distribuicao.rename(columns={"Municipio": "Município"}, inplace=True)
distribuicao.rename(columns={"Qtd-Doses-Distribuidas": 
                             "Doses Distribuídas"}, inplace=True)


# Agrupando outro dataframe ao principal, para inserir informações adicionais
df = vacinometro.merge(distribuicao, left_on=['Município'], 
                   right_on=['Município'], how='inner', 
                   suffixes=['_vacinometro', '_distribuicao'])


# Reformatando os valores da coluna "Município", que estavam em caixa alta
s = df['Município'].str.title()
df.loc[:, 'Município'] = pd.Series(s, index=df.index, name='Município')


,Município,1ª Dose,2ª Dose,3ª Dose,Dose Única,Doses Distribuídas
0,Adamantina,29365,22101,816,878,54136
1,Adolfo,3664,2910,98,77,6044
2,Aguaí,26927,19971,435,1042,52634
3,Agudos,31231,22319,299,873,57594
4,Alambari,4624,3381,55,171,8389
...,...,...,...,...,...,...
640,Águas De São Pedro,3402,2676,164,58,7293
641,Álvares Florence,3288,2662,73,80,6579
642,Álvares Machado,19603,14169,235,600,36980
643,Álvaro De Carvalho,5331,2896,55,107,8526


**Evolução da aplicação de doses por Data** *(não é a fonte principal, mas poderá ser útil para a criação de insights no futuro como, por exemplo, a relação do aumento da vacinação com a diminuição da transmissão e letalidade da COVID-19)*

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import sys
import os
import glob
import time
import re


# Automação da coleta dos dataframes pelo selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
download_dir = "/content/evolucao"
prefs = {'download.default_directory' : download_dir}

options_chrome = Options()
options_chrome.add_argument("start-maximized")
options_chrome.add_argument("disable-infobars")
options_chrome.add_argument("--disable-extensions")
options_chrome.add_argument('--headless')
options_chrome.add_argument('--no-sandbox')
options_chrome.add_argument('--disable-dev-shm-usage')
options_chrome.add_experimental_option('prefs', prefs)
desired = options_chrome.to_capabilities()
desired['loggingPrefs'] = { 'performance': 'ALL'}

navegador = webdriver.Chrome('chromedriver', options=options_chrome, 
                             desired_capabilities=desired)

navegador.get("https://www.saopaulo.sp.gov.br/planosp/simi/dados-abertos/")
botao3 = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[19]/h3").click()
elemento3 = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[19]/div/ul/li/div/a")
href3 = elemento3.get_attribute("href")
navegador.get(href3)

time.sleep(15)


# Automatação do reconhecimento de novos dados pelo Pandas
list_of_files = glob.glob('/content/evolucao/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
csv = os.path.abspath(latest_file)


# Leitura do Dataframe pelo pandas
df2 = pd.read_csv(csv, sep=';')


# Criando novas colunas de tal forma que não tenha linhas duplicadas
col = 'Dose'
condition1 = [df2[col] == '1° DOSE']
condition2 = [df2[col] == '2° DOSE']
condition3 = [df2[col] == '3º DOSE/ADICIONAL']
condition4 = [df2[col] == 'UNICA']
choice = [df2["Contagem de Dose"]]
                       
df2["1ª Dose"] = np.select(condition1, choice, default=np.nan)
df2["2ª Dose"] = np.select(condition2, choice, default=np.nan)
df2["3ª Dose"] = np.select(condition3, choice, default=np.nan)
df2["Dose Única"] = np.select(condition4, choice, default=np.nan)

df2 = df2.groupby('Dia de Data Registro Vacina').agg({'1ª Dose':
                                                      'first', '2ª Dose':'first', 
                                                      '3ª Dose':'first', 
                                                      'Dose Única':'first'}).reset_index()


# Ajustando os missing values encontrados para converter em int64:
df2["1ª Dose"] = df2["1ª Dose"].fillna(0)
df2["2ª Dose"] = df2["2ª Dose"].fillna(0)
df2["3ª Dose"] = df2["3ª Dose"].fillna(0)
df2["Dose Única"] = df2["Dose Única"].fillna(0)


# Ajustando o Datatype de colunas
df2["1ª Dose"] = df2["1ª Dose"].astype(int)
df2["2ª Dose"] = df2["2ª Dose"].astype(int)
df2["3ª Dose"] = df2["3ª Dose"].astype(int)
df2["Dose Única"] = df2["Dose Única"].astype(int)


# Alterando nome de colunas
df2.rename(columns={"Dia de Data Registro Vacina": "Data de Registro"}, 
           inplace=True)


,Data de Registro,1ª Dose,2ª Dose,3ª Dose,Dose Única
0,1 de abril de 2021,91072,43424,0,0
1,1 de agosto de 2021,16646,7525,0,164
2,1 de fevereiro de 2021,56138,48,0,0
3,1 de julho de 2021,332174,11956,0,33477
4,1 de junho de 2021,150175,56053,0,1
...,...,...,...,...,...
255,9 de julho de 2021,245744,24971,0,87589
256,9 de junho de 2021,177093,14212,0,1
257,9 de maio de 2021,11472,2918,0,0
258,9 de março de 2021,69340,50712,0,0


**Estatísticas Gerais** *(útil para utilizar nos painéis centrais do site)*

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import sys
import os
import glob
import time


# Automação da coleta dos dataframes pelo selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
download_dir = "/content/estatisticas_gerais"
prefs = {'download.default_directory' : download_dir}

options_chrome = Options()
options_chrome.add_argument("start-maximized")
options_chrome.add_argument("disable-infobars")
options_chrome.add_argument("--disable-extensions")
options_chrome.add_argument('--headless')
options_chrome.add_argument('--no-sandbox')
options_chrome.add_argument('--disable-dev-shm-usage')
options_chrome.add_experimental_option('prefs', prefs)
desired = options_chrome.to_capabilities()
desired['loggingPrefs'] = { 'performance': 'ALL'}

navegador = webdriver.Chrome('chromedriver', options=options_chrome, 
                             desired_capabilities=desired)

navegador.get("https://www.saopaulo.sp.gov.br/planosp/simi/dados-abertos/")
botao4 = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[20]/h3").click()
elemento4 = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[20]/div/ul/li/div/a")
href4 = elemento4.get_attribute("href")
navegador.get(href4)

time.sleep(15)


# Automatação do reconhecimento de novos dados pelo Pandas
list_of_files = glob.glob('/content/estatisticas_gerais/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
csv = os.path.abspath(latest_file)


# Leitura do dataframe pelo pandas
df3 = pd.read_csv(csv, sep=';')


# Renomeando linhas
df3.at[1, 'Nomes de medida'] = '3ª Dose'
df3.at[0, 'Nomes de medida'] = 'Doses Aplicadas'


# Renomeando colunas
df3 = df3.rename(columns={'Valores de medida': 'Total'})
df3 = df3.rename(columns={'Nomes de medida': 'Dado'})


# Alterando a ordem das linhas
df3 = df3.sort_values(['Dado'], ascending=[True])


,Dado,Total
4,1ª Dose,36760944
3,2ª Dose,25733772
1,3ª Dose,502946
2,Dose Única,1157247
0,Doses Aplicadas,64154909


# Leitos de UTI e Enfermaria


**Leitos de UTI e Enfermagem por Diretoria Regional de Saúde - DRS** *(fonte principal para dados de leitos dos Municípios e do Estado)*

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import sys
import time
import glob
import os


# Automação da coleta dos dataframes pelo selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
download_dir = "/content/leitos_uti"
prefs = {'download.default_directory' : download_dir}

options_chrome = Options()
options_chrome.add_argument("start-maximized")
options_chrome.add_argument("disable-infobars")
options_chrome.add_argument("--disable-extensions")
options_chrome.add_argument('--headless')
options_chrome.add_argument('--no-sandbox')
options_chrome.add_argument('--disable-dev-shm-usage')
options_chrome.add_experimental_option('prefs', prefs)
desired = options_chrome.to_capabilities()
desired['loggingPrefs'] = { 'performance': 'ALL'}

navegador = webdriver.Chrome('chromedriver', options=options_chrome, 
                             desired_capabilities=desired)

navegador.get("https://www.saopaulo.sp.gov.br/planosp/simi/dados-abertos/")
botao1 = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[7]/h3").click()
elemento = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[7]/div/ul/li[1]/div/a")
href = elemento.get_attribute("href")
navegador.get(href)

time.sleep(15)


# Automatação do reconhecimento de novos dados pelo Pandas
list_of_files = glob.glob('/content/leitos_uti/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
csv = os.path.abspath(latest_file)


# Leitura pelo Pandas do arquivo csv
df = pd.read_csv(csv, sep=';')


# Cleaning Data

# Deletando colunas sem dados relevantes
df.drop('internacoes_7d_l', axis=1, inplace=True)
df.drop('leitos_pc', axis=1, inplace=True)


# Alterando a "," pelo "." para a conversão em float sem gerar valores NaN
df['pacientes_uti_mm7d'] = df['pacientes_uti_mm7d'].str.replace(',','.')
df['total_covid_uti_mm7d'] = df['total_covid_uti_mm7d'].str.replace(',','.')
df['ocupacao_leitos'] = df['ocupacao_leitos'].str.replace(',','.')
df['internacoes_7v7'] = df['internacoes_7v7'].str.replace(',','.')
df['ocupacao_leitos_ultimo_dia'] = df['ocupacao_leitos_ultimo_dia'].str.replace(',','.')
df['pacientes_enf_mm7d'] = df['pacientes_enf_mm7d'].str.replace(',','.')
df['total_covid_enf_mm7d'] = df['total_covid_enf_mm7d'].str.replace(',','.')


# Alterando o Datatype de algumas colunas
df['pacientes_uti_mm7d'] = df['pacientes_uti_mm7d'].apply(pd.to_numeric, downcast='float', errors='coerce')
df['total_covid_uti_mm7d'] = df['total_covid_uti_mm7d'].apply(pd.to_numeric, downcast='float', errors='coerce')
df['ocupacao_leitos'] = df['ocupacao_leitos'].apply(pd.to_numeric, downcast='float', errors='coerce')
df['internacoes_7v7'] = df['internacoes_7v7'].apply(pd.to_numeric, downcast='float', errors='coerce')
df['ocupacao_leitos_ultimo_dia'] = df['ocupacao_leitos_ultimo_dia'].apply(pd.to_numeric, downcast='float', errors='coerce')
df['pacientes_enf_mm7d'] = df['pacientes_enf_mm7d'].apply(pd.to_numeric, downcast='float', errors='coerce')
df['total_covid_enf_mm7d'] = df['total_covid_enf_mm7d'].apply(pd.to_numeric, downcast='float', errors='coerce')


# Delimitando as casas decimais em colunas float:
df['pacientes_uti_mm7d'] = df['pacientes_uti_mm7d'].round(decimals=2)
df['total_covid_uti_mm7d'] = df['total_covid_uti_mm7d'].round(decimals=2)
df['ocupacao_leitos'] = df['ocupacao_leitos'].round(decimals=2)
df['internacoes_7v7'] = df['internacoes_7v7'].round(decimals=2)
df['ocupacao_leitos_ultimo_dia'] = df['ocupacao_leitos_ultimo_dia'].round(decimals=2)
df['pacientes_enf_mm7d'] = df['pacientes_enf_mm7d'].round(decimals=2)
df['total_covid_enf_mm7d'] = df['total_covid_enf_mm7d'].round(decimals=2)


# Renomeando colunas
df.rename(columns={"ocupacao_leitos": "mm7d da Ocupação dos leitos de UTI e Enfermaria (%)"}, inplace=True)
df.rename(columns={"datahora": "Data"}, inplace=True)
df.rename(columns={"total_covid_uti_mm7d": "mm7d do Total de leitos de UTI destinados à Covid"}, inplace=True)
df.rename(columns={"internacoes_7d": "Nº de novas internações nos últimos 7 dias"}, inplace=True)
df.rename(columns={"internacoes_7v7": "Variação no nº de novas internações (%)"}, inplace=True)
df.rename(columns={"pacientes_uti_ultimo_dia": "Pacientes em tratamento na UTI"}, inplace=True)
df.rename(columns={"total_covid_uti_ultimo_dia": "Total de leitos de UTI destinados à Covid"}, inplace=True)
df.rename(columns={"ocupacao_leitos_ultimo_dia": "Ocupação dos leitos de UTI e Enfermaria (%)"}, inplace=True)
df.rename(columns={"internacoes_ultimo_dia": "Novos casos de internações (UTI e Enfermaria)"}, inplace=True)
df.rename(columns={"total_covid_enf_mm7d": "mm7d do Total de leitos de Enfermaria destinados à Covid"}, inplace=True)
df.rename(columns={"pacientes_enf_ultimo_dia": "Pacientes em tratamento na Enfermaria"}, inplace=True)
df.rename(columns={"total_covid_enf_ultimo_dia": "Total de leitos de Enfermaria destinados à Covid"}, inplace=True)
df.rename(columns={"nome_drs": "Departamento Regional de Saúde"}, inplace=True)
df.rename(columns={"pacientes_uti_mm7d": "mm7d do Nº de pacientes em tratamento na UTI"}, inplace=True)
df.rename(columns={"pop": "População"}, inplace=True)
df.rename(columns={"pacientes_enf_mm7d": "mm7d do Nº de pacientes em tratamento na Enfermaria"}, inplace=True)


# Transformando a coluna de Data de Notificação em Datetime type
df.loc[:, 'Data'] = pd.Series(pd.to_datetime(df['Data'], dayfirst=True, 
                                             format='%d/%m/%Y', errors='coerce'), 
                                             name='Data')


# Ordenando o dataframe por data e resetando o index
df = df.reset_index(drop=True)

df = df.sort_values(['Data'], ascending=True)

df = df[[c for c in df.columns if c not in ['index']]]
if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
	df = df.to_frame(index=False)
df = df.reset_index().drop('index', axis=1, errors='ignore')


,Data,Departamento Regional de Saúde,mm7d do Nº de pacientes em tratamento na UTI,mm7d do Total de leitos de UTI destinados à Covid,mm7d da Ocupação dos leitos de UTI e Enfermaria (%),População,Nº de novas internações nos últimos 7 dias,Variação no nº de novas internações (%),Pacientes em tratamento na UTI,Total de leitos de UTI destinados à Covid,Ocupação dos leitos de UTI e Enfermaria (%),Novos casos de internações (UTI e Enfermaria),mm7d do Nº de pacientes em tratamento na Enfermaria,mm7d do Total de leitos de Enfermaria destinados à Covid,Pacientes em tratamento na Enfermaria,Total de leitos de Enfermaria destinados à Covid
0,2020-05-19,Município de São Paulo,2268.43,2868.00,79.09,11869660,5049,6.79,2402,2990,80.33,608,3447.14,4980.86,3498,5010
1,2020-05-19,Estado de São Paulo,4180.57,6323.00,66.12,44639899,10636,13.61,4428,6534,67.77,1328,6626.14,13124.00,6795,13337
2,2020-05-19,DRS 17 Taubaté,84.00,206.86,40.61,2489629,311,24.90,87,211,41.23,46,128.14,570.43,154,568
3,2020-05-19,DRS 16 Sorocaba,79.29,165.71,47.85,2461760,241,34.64,94,163,57.67,14,137.29,432.57,136,448
4,2020-05-19,DRS 15 São José do Rio Preto,41.57,199.43,20.84,1570421,178,14.84,35,195,17.95,27,85.71,328.86,99,327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11633,2021-10-06,Grande SP Leste,107.57,352.43,30.52,2977781,200,-4.76,108,327,33.03,39,138.00,420.29,152,404
11634,2021-10-06,Município de São Paulo,927.00,1873.14,49.49,11869660,1538,-2.66,913,1845,49.49,229,1107.86,2774.71,1122,2718
11635,2021-10-06,DRS 17 Taubaté,87.86,369.71,23.76,2489629,231,14.36,85,359,23.68,41,103.43,626.57,105,607
11636,2021-10-06,DRS 06 Bauru,48.00,255.71,18.77,1741281,79,-11.24,46,252,18.25,17,51.29,563.14,51,593


**Leitos de UTI e Enfermagem por Unidade Hospitalar Pública** *(não é prioritário, mas pode ser usado para complementar as informações dos municípios como, por exemplo, mapear essas unidades)*

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import sys
import time
import glob
import os


# Automação da coleta dos dataframes pelo selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
download_dir = "/content/unidade_hospitalar"
prefs = {'download.default_directory' : download_dir}

options_chrome = Options()
options_chrome.add_argument("start-maximized")
options_chrome.add_argument("disable-infobars")
options_chrome.add_argument("--disable-extensions")
options_chrome.add_argument('--headless')
options_chrome.add_argument('--no-sandbox')
options_chrome.add_argument('--disable-dev-shm-usage')
options_chrome.add_experimental_option('prefs', prefs)
desired = options_chrome.to_capabilities()
desired['loggingPrefs'] = { 'performance': 'ALL'}

navegador = webdriver.Chrome('chromedriver', options=options_chrome, 
                             desired_capabilities=desired)

navegador.get("https://www.saopaulo.sp.gov.br/planosp/simi/dados-abertos/")
botao = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[8]/h3").click()
elemento = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[8]/div/ul/li[1]/div/a")
href = elemento.get_attribute("href")
navegador.get(href)

time.sleep(15)


# Automatação do reconhecimento de novos dados pelo Pandas
list_of_files = glob.glob('/content/unidade_hospitalar/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
csv = os.path.abspath(latest_file)


# leitura do dataframe pelo pandas
df = pd.read_csv(csv, sep=';') 


# Renomeando colunas
df = df.rename(columns={'Municipio': 'Município'})
df = df.rename(columns={'Nome Fantasia': 'Unidade Hospitalar'})
df = df.rename(columns={'Leitos Ocupados / Enfermaria': 'Leitos de Enfermaria Ocupados'})
df = df.rename(columns={'Leitos Ocupados / UTI': 'Leitos de UTI Ocupados'})
df = df.rename(columns={'DRS Nome': 'Departamento Regional de Saúde'})


# Ajustando os valores das colunas que estão em caixa alta
s = df['Município'].str.title()
df.loc[:, 'Município'] = pd.Series(s, index=df.index, name='Município')
s = df['Unidade Hospitalar'].str.title()
df.loc[:, 'Unidade Hospitalar'] = pd.Series(s, index=df.index, 
                                            name='Unidade Hospitalar')


,Departamento Regional de Saúde,Município,Unidade Hospitalar,Leitos de Enfermaria Ocupados,Leitos de UTI Ocupados
0,Araçatuba,Araçatuba,Hospital Municipal Da Mulher Dr Joao Luis J Ro...,2,0
1,Araçatuba,Mirandópolis,Unid Int Saude Dr Oswaldo B Faria De Mirandopolis,4,0
2,Araraquara,Américo Brasiliense,Hospital Estadual Américo Brasiliense,1,1
3,Araraquara,Américo Brasiliense,Unidade Hospitalar Dr. José Nigro Neto,1,0
4,Araraquara,Araraquara,Fundação Municipal Irene Siqueira Alves Hospital,0,1
...,...,...,...,...,...
162,Taubaté,Jacareí,Upa Iii Dr Thelmo De Almeida Cruz,1,0
163,Taubaté,São José Dos Campos,Hospital Municipal Dr. José Carvalho Florence,21,13
164,Taubaté,São José Dos Campos,Hospital Regional De São José Dos Campos,0,3
165,Taubaté,Taubaté,Hospital De Campanha Covid 19 Municipio De Tau...,4,0


# Isolamento Social

**Índice de Isolamento Social por Município e por Data** *(principal fonte de dados de Isolamento Social)*

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import sys
import time
import glob
import os
import datetime as dt
from datetime import datetime


# Automação da coleta dos dataframes
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
download_dir = "/content/isolamento"
prefs = {'download.default_directory' : download_dir}

options_chrome = Options()
options_chrome.add_argument("start-maximized")
options_chrome.add_argument("disable-infobars")
options_chrome.add_argument("--disable-extensions")
options_chrome.add_argument('--headless')
options_chrome.add_argument('--no-sandbox')
options_chrome.add_argument('--disable-dev-shm-usage')
options_chrome.add_experimental_option('prefs', prefs)
desired = options_chrome.to_capabilities()
desired['loggingPrefs'] = { 'performance': 'ALL'}

navegador = webdriver.Chrome('chromedriver', options=options_chrome, 
                             desired_capabilities=desired)

navegador.get("https://www.saopaulo.sp.gov.br/planosp/simi/dados-abertos/")
botao1 = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[13]/h3").click()
elemento = navegador.find_element_by_xpath("/html/body/section[4]/div/div/article[13]/div/ul/li/div/a")
href = elemento.get_attribute("href")
navegador.get(href)

time.sleep(15)


# Automação do reconhecimento de novos dados pelo Pandas
list_of_files = glob.glob('/content/isolamento/*.csv')
latest_file = max(list_of_files, key=os.path.getctime)
csv = os.path.abspath(latest_file)


# Leitura pelo Pandas do arquivo csv
df = pd.read_csv(csv, sep=';', parse_dates=['Data'])


# Cleaning Data

# Renomeando colunas
df.rename(columns={"Município1": "Município"}, inplace=True)
df.rename(columns={"Código Município IBGE": "codigo_ibge"}, inplace=True)
df.rename(columns={"População estimada (2020)": "População"}, inplace=True)
df.rename(columns={"Média de Índice De Isolamento": "Índice de Isolamento (%)"}, inplace=True)
df.rename(columns={"Data": "data_reg"}, inplace=True)


# Deletando colunas sem dados relevantes
df.drop('UF1', axis=1, inplace=True)


# Ajustando os valores da coluna "Município" que está em caixa alta
s = df['Município'].str.title()
df.loc[:, 'Município'] = pd.Series(s, name='Município')


# Retirando a "%" para a conversão do índice em int64 sem gerar valores null
df['Índice de Isolamento (%)'] = df['Índice de Isolamento (%)'].str.replace('%','')
df["Índice de Isolamento (%)"] = df["Índice de Isolamento (%)"].astype(int)


# Usando delimitadores para separar a Data e o dia da semana
df.loc[:, 'Dia da Semana'] = df['data_reg']
df['Dia da Semana'] = df['Dia da Semana'].str.split(',', expand=True)
df['data_reg'] = df['data_reg'].str.replace('segunda-feira, ','')
df['data_reg'] = df['data_reg'].str.replace('terça-feira, ','')
df['data_reg'] = df['data_reg'].str.replace('quarta-feira, ','')
df['data_reg'] = df['data_reg'].str.replace('quinta-feira, ','')
df['data_reg'] = df['data_reg'].str.replace('sexta-feira, ','')
df['data_reg'] = df['data_reg'].str.replace('sábado, ','')
df['data_reg'] = df['data_reg'].str.replace('domingo, ','')


# Gerando dois dataframes adjacentes para atribuir o ano correto em cada Data

# Dataframe de dados 2020
duplicates = df.duplicated(['data_reg','Município'], keep='last')
df_last = df[~duplicates]
df_last.loc[:, 'Data'] = df['data_reg'] + '/2020'

df_last.loc[:, 'Data'] = pd.Series(pd.to_datetime(df_last['Data'], dayfirst=True, 
                                                   format='%d/%m/%Y', errors='coerce'), 
                                    name='Data')                                  

df_last = df_last.reset_index(drop=True)

df_last = df_last.sort_values(['Data'], ascending=True)

start = df_last['Data'].searchsorted(dt.datetime.strptime('2020-03-17', '%Y-%m-%d'))
end = df_last['Data'].searchsorted(dt.datetime.strptime('2021-01-01', '%Y-%m-%d'))
df_last = df_last.iloc[start:end]

df_last = df_last[[c for c in df_last.columns if c not in ['index']]]
if isinstance(df_last, (pd.DatetimeIndex, pd.MultiIndex)):
	df_last = df_last.to_frame(index=False)
df_last = df_last.reset_index().drop('index', axis=1, errors='ignore')

# Dataframe de dados 2021
duplicates = df.duplicated(['data_reg','Município'], keep='first')
df_first = df[~duplicates]
df_first.loc[:, 'Data'] = df['data_reg'] + '/2021'
df_first.loc[:, 'Data'] = pd.Series(pd.to_datetime(df_first['Data'], dayfirst=True, 
                                                   format='%d/%m/%Y', errors='coerce'), 
                                    name='Data',)                                                               

df_first = df_first.reset_index(drop=True)

df_first = df_first.sort_values(['Data'], ascending=True)

yesterday = (datetime.now() - dt.timedelta(days=1)).strftime("%Y-%m-%d")
start = df_first['Data'].searchsorted(dt.datetime.strptime('2021-01-01', '%Y-%m-%d'))
end = df_first['Data'].searchsorted(dt.datetime.strptime(yesterday, '%Y-%m-%d'))
df_first = df_first.iloc[start:end]

df_first = df_first[[c for c in df_first.columns if c not in ['index']]]
if isinstance(df_first, (pd.DatetimeIndex, pd.MultiIndex)):
	df_first = df_first.to_frame(index=False)
df_first = df_first.reset_index().drop('index', axis=1, errors='ignore')


# Agrupando os dataframes com as datas e fazendo os últimos ajustes:
df = pd.concat([df_first,df_last])

df = df.sort_values(['Data'], ascending=True)

duplicates = df.duplicated(['data_reg','Município','Dia da Semana'], keep='last')
df = df[~duplicates]

df.drop('data_reg', axis=1, inplace=True)

df = df[[c for c in df.columns if c not in ['index']]]
if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
	df = df.to_frame(index=False)
df = df.reset_index().drop('index', axis=1, errors='ignore')


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Município,codigo_ibge,População,Índice de Isolamento (%),Dia da Semana,Data
0,Embu-Guaçu,3515103,69901,40,terça-feira,2020-03-17
1,Ubatuba,3555406,91824,54,terça-feira,2020-03-17
2,Várzea Paulista,3556503,123071,37,terça-feira,2020-03-17
3,Boituva,3507001,62170,41,terça-feira,2020-03-17
4,Leme,3526704,104346,48,terça-feira,2020-03-17
...,...,...,...,...,...,...
79497,Jaguariúna,3524709,58722,40,terça-feira,2021-10-05
79498,Jandira,3525003,126356,36,terça-feira,2021-10-05
79499,Itatiba,3523404,122581,34,terça-feira,2021-10-05
79500,Itu,3523909,175568,33,terça-feira,2021-10-05
